In [6]:
from flask import Flask, redirect, url_for, render_template, request
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from skimage.transform import resize
import tensorflow as tf
import os
import cv2

In [7]:
app = Flask(__name__)

In [8]:
@app.route("/")
def home():
    return render_template("index.html")

In [9]:
@app.route("/explanation", methods = ['POST'])
def explanation():
    image = request.files['filename']
    image.save(os.path.join(os.getcwd(),"static","image.jpg"))
    image = cv2.imread('static/image.jpg')
    explainer = lime_image.LimeImageExplainer()
    model = tf.keras.models.load_model('model/')
    image = resize(image, (150, 150))
    
    explanation = explainer.explain_instance(image, model.predict, top_labels=2, hide_color=0, num_samples=1000)
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
    plt.imsave(os.path.join(os.getcwd(),"static","figure1.png"),mark_boundaries(temp , mask))
    
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    plt.imsave(os.path.join(os.getcwd(),"static","figure2.png"),mark_boundaries(temp , mask))
    
    return render_template("explanation.html")

In [10]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2021 15:49:07] "GET / HTTP/1.1" 200 -
[2021-08-01 15:49:11,287] ERROR in app: Exception on /explanation [POST]
Traceback (most recent call last):
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/h